In [1]:
import os
import subprocess
import pandas as pd
import shutil

In [27]:
class Runner:
    def __init__(self, name, expression_dir, args, backend=None):
        """
        This class handles the code generation and execution of the variant codes. 
        The generated event data can be obtained as a pandas dataframe.
        
        Requirements:
        
        It is assumed that there exists a script file that generates variant codes
        for a given oopoerand sizes. The operand sizes are input as command line args
            e.g. run of script file: python generate.py 10 10 10 10 12
        
        After running the script file, inside the folder "experiments", which is in 
        the same directory as the script file, an "argument folder" is generated, 
        which contains the case_table, event_meta_table (i.e, the event table without actual run times)
        ,and a runner script as shown in the expample below:
            e.g. experiment/10_10_10_10_12/
                    case_table.csv
                    event_meta_table.csv
                    runner.jl 
        
        'runner.jl' is the script that runs the experiments and generates a log file 'run_times.txt' (which is the
        event table with actual run times) in the "arguments folder"
        
        
        INPUT:
        
        name: Experiment name
        script_path: Path to the script file that generates variants
        args: operand sizes (or arguments to the script file)
        
        USECASE:
        If the behavior of the script is as said in the requirements, this class can 
        call the scipt file and collects the eventlogs as a pandas dataframe, and 
        if needed, can also clean the generated folders.
        
        """
        self.name = name
        self.expression_dir = expression_dir
        
        self.script_path = os.path.join(self.expression_dir, "generate-variants-linnea.py")
        self.args = args
        self.call = ["python", self.script_path] + self.args
        self.args_dir = os.path.join(self.expression_dir,
                                   "experiments",
                                  "_".join(self.args))
        
    
    def generate_experiments(self, backend=False):  
        """
        generates experiments for a given set of valid arguments
        that can be given as input to the script file.
            e.g. in,  python generate.py 10 10 10 10 12
            ['10','10','10','10','12'] would be the argument list.
            
        Output: Return code == 0 implies successful completion 
        """
        
        if not backend:
            completed_proccess = subprocess.run(self.call)
        #else:
            
        return completed_proccess.returncode;
    
       
    def run_experiments(self,backend=False):
        """
        executes the runner file, which generates run_times.txt
        """         
        if os.path.exists(self.exp_dir):
            if not backend:
                runner_path = os.path.join(self.exp_dir,"runner.jl")
                print("Running Experiments locally")
                completed_proccess = subprocess.run(["julia", runner_path])
                if completed_proccess.returncode == 0:
                    print("Experiments completed locally")
                    return 1 # Ran experiment  
            else:
                runner_path = os.path.join(self.exp_dir,"runner_on_backend.sh")
                if os.path.exists(runner_path):
                    print("Submitted job to cluster")
                    completed_proccess = subprocess.run(["sbatch", runner_path])
                    return 2 # Submitted job to a backend
        return -1
                
    def get_case_table(self):
        """get case table"""
        if os.path.exists(self.exp_dir):
            return self.read_log(os.path.join(self.exp_dir,"case_table.csv"))
        return -1
    
    def get_event_meta_table(self):
        """get event table without actual execution times."""
        if os.path.exists(self.exp_dir):
            return self.read_log(os.path.join(self.exp_dir,"event_meta_table.csv"))
        return -1
    
    def get_event_runtime_table(self):    
        """get event table with actual execution times."""
        if os.path.exists(self.exp_dir):
            return self.read_log(os.path.join(self.exp_dir,"run_times.txt"))
        return -1
    
    def get_all_tables(self, meta=True):
        """get all tables"""
        case_table = self.get_case_table()
        event_meta_table = None
        if meta:
            event_meta_table = self.get_event_meta_table()
        event_runtime_table = self.get_event_runtime_table()
        return (case_table,event_meta_table,event_runtime_table)
        
    def read_log(self, log_path):
        if os.path.exists(log_path):
            df = pd.read_csv(log_path,sep=';')
            return df
        return -1

    def isGenerated(self):
        if os.path.exists(self.exp_dir):
            return True
        return False
    
    def isRun(self):
        if os.path.exists(os.path.join(self.exp_dir, "run_times.txt")):
            return True
        return False
    
    def clean(self):
        """remove arguments folder"""
        if os.path.exists(self.exp_dir):
            shutil.rmtree(self.exp_dir)
        else:
            return -1
            
            

In [28]:
args = ["10","110","120","130","501"]
runner = Runner("Matrix-Chain", "../Matrix-Chain-4/variants-linnea/generate-variants-linnea.py",args)

In [29]:
ret = runner.generate_run_experiments(bRun=True, bGenerate=True)

New solution:.............2.72e+06
No further generation steps possible.
----------------------------------
Number of nodes:                 8
Solution nodes:                  1
Data:                     1.38e+05
Best solution:            2.72e+06
Intensity:                    19.7
Number of algorithms:            6
Generated Variants.
Running Experiments
Experiments completed


In [30]:
ret

1

In [31]:
ct,et,rt = runner.get_all_tables(meta=True)

In [32]:
ct

,case:concept:name,case:flops,case:num_kernels
0,algorithm1,14900000.0,3
1,algorithm5,29100000.0,3
2,algorithm4,29100000.0,3
3,algorithm0,2720000.0,3
4,algorithm3,18000000.0,3
5,algorithm2,17900000.0,3


In [33]:
et

,case:concept:name,concept:name,concept:flops,concept:kernel,concept:operation,timestamp:start
0,algorithm1,gemm_3.12e+05,312000.0,"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",tmp2 = (B C),2022-06-13 16:08:14.311588
1,algorithm1,gemm_2.86e+05,286000.0,"gemm!('N', 'N', 1.0, ml0, ml4, 0.0, ml5)",tmp4 = (A tmp2),2022-06-13 16:08:15.311588
2,algorithm1,gemm_1.43e+07,14300000.0,"gemm!('N', 'N', 1.0, ml5, ml3, 0.0, ml6)",tmp6 = (tmp4 D),2022-06-13 16:08:16.311588
3,algorithm5,gemm_2.64e+05,264000.0,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",tmp1 = (A B),2022-06-13 16:14:54.311588
4,algorithm5,gemm_1.56e+07,15600000.0,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml5)",tmp3 = (C D),2022-06-13 16:14:55.311588
5,algorithm5,gemm_1.32e+07,13200000.0,"gemm!('N', 'N', 1.0, ml4, ml5, 0.0, ml6)",tmp6 = (tmp1 tmp3),2022-06-13 16:14:56.311588
6,algorithm4,gemm_1.56e+07,15600000.0,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml4)",tmp3 = (C D),2022-06-13 16:13:14.311588
7,algorithm4,gemm_2.64e+05,264000.0,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml5)",tmp1 = (A B),2022-06-13 16:13:15.311588
8,algorithm4,gemm_1.32e+07,13200000.0,"gemm!('N', 'N', 1.0, ml5, ml4, 0.0, ml6)",tmp6 = (tmp1 tmp3),2022-06-13 16:13:16.311588
9,algorithm0,gemm_3.12e+05,312000.0,"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",tmp2 = (B C),2022-06-13 16:06:34.311588


In [34]:
rt

,case:concept:name,concept:name,concept:flops,concept:operation,concept:kernel,timestamp:start,timestamp:end
0,algorithm1,gemm_3.12e+05,312000.0,tmp2 = (B C),"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.655129e+09,1.655129e+09
1,algorithm1,gemm_2.86e+05,286000.0,tmp4 = (A tmp2),"gemm!('N', 'N', 1.0, ml0, ml4, 0.0, ml5)",1.655129e+09,1.655129e+09
2,algorithm1,gemm_1.43e+07,14300000.0,tmp6 = (tmp4 D),"gemm!('N', 'N', 1.0, ml5, ml3, 0.0, ml6)",1.655129e+09,1.655129e+09
3,algorithm5,gemm_2.64e+05,264000.0,tmp1 = (A B),"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",1.655129e+09,1.655129e+09
4,algorithm5,gemm_1.56e+07,15600000.0,tmp3 = (C D),"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml5)",1.655129e+09,1.655129e+09
5,algorithm5,gemm_1.32e+07,13200000.0,tmp6 = (tmp1 tmp3),"gemm!('N', 'N', 1.0, ml4, ml5, 0.0, ml6)",1.655129e+09,1.655129e+09
6,algorithm4,gemm_1.56e+07,15600000.0,tmp3 = (C D),"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml4)",1.655129e+09,1.655129e+09
7,algorithm4,gemm_2.64e+05,264000.0,tmp1 = (A B),"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml5)",1.655129e+09,1.655129e+09
8,algorithm4,gemm_1.32e+07,13200000.0,tmp6 = (tmp1 tmp3),"gemm!('N', 'N', 1.0, ml5, ml4, 0.0, ml6)",1.655129e+09,1.655129e+09
9,algorithm0,gemm_3.12e+05,312000.0,tmp2 = (B C),"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.655129e+09,1.655129e+09


In [35]:
runner.clean()